In [1]:
from gpt import GPTConfig, GPT, generate
import torch
import os
from transformers import AutoTokenizer


In [2]:
def print_leaderboard_and_get_best_model():
    candidates = []
    for expt in os.listdir('logs'):
        if 'smoke' in expt:
            continue
        best_loss_for_expt = 10000
        best_line = ''

        for line in open(f'logs/{expt}/log.txt').readlines():

            if 'val' in line and 'byte loss' in line:
                def get_byte_loss():
                    after_bl_str = line.split('byte loss')[1]            
                    try:                
                        byte_loss = float(after_bl_str.split(' ')[1])
                    except ValueError as e:
                        byte_loss = 100000
                    return byte_loss
                
                byte_loss = get_byte_loss()

                if byte_loss < best_loss_for_expt:
                    best_loss_for_expt = byte_loss
                    best_line = line
        
        if best_loss_for_expt < 10000:
            candidates.append((best_loss_for_expt, expt, best_line))

    candidates = sorted(candidates, key=lambda x: x[0])
    for best_loss, expt, line in candidates:
        print(expt.ljust(50), f'{best_loss:.3} {line.strip()}')

    best_expt = candidates[0][1]
    best_model_paths = [model_path for model_path in os.listdir(f'logs/{best_expt}') if 'model' in model_path]
    best_model_path = sorted(best_model_paths, key=lambda x: x.split('_')[1])[-1]
    return best_expt, best_model_path

best_expt, best_model_path = print_leaderboard_and_get_best_model()


16l_16h_512d_quick                                 0.843 step 499 | val loss 3.4470 | byte loss 0.8426 | ds 90.2s


In [3]:
choosen_model = f'logs/{best_expt}/{best_model_path}'
# choosen_model = f'logs/16l_16h_512d_quick/model_00323.pt'
full_checkpoint = torch.load(choosen_model)
config = full_checkpoint['config']
m = GPT(config)

def remove_orig_mod_prefix(state_dict):
    return {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}

m.load_state_dict(remove_orig_mod_prefix(full_checkpoint['model']))
m.to('cuda')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 512)
    (wpe): Embedding(512, 512)
    (h): ModuleList(
      (0-15): 16 x Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=1024, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=1024, out_features=512, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=8192, bias=False)
)

In [4]:
enc = AutoTokenizer.from_pretrained('activated-ai/tiny-stories-8k-tokenizer')

In [5]:
generate(m, enc, "Lily went to the park and saw a friendly dog.", 255, 4)

sample 0: Lily went to the park and saw a friendly dog. They were playing in their eyes. They gave them a bird and then they played together in the man, but they played.
"Come, you for lunch!" Mia said.

"Maybe we go outside and put on its room. We are a small bear.


"We were sad. But I wish we need to be careful," said. The deer said. The bear. The bird thought. They ran to the man and put the dog.
Sara smiled. They also had to the sun back to say sorry for more. They gave the lady for the bear.

"I'm just there. They can play. And we are my ball out. Do you like the other," they feel glad they are not nice and good.
sample 1: Lily went to the park and saw a friendly dog.
She reached the tree and ran towards the tree started to his car.
"Look, you need?" Mom said. "Yes, and Ben with them and Ben.


Her mom.

Anna and Max saw Tom did that they were playing with their toys and took out. They looked at the blocks and the slide in their eyes. They pulled for them by the rain. They were h